In [1]:
import pandas as pd

steam_data_filepath = r"..\data\raw\Steam_2024_bestRevenue_1500.csv"

steam_df = pd.read_csv(steam_data_filepath)


In [2]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1500 non-null   object 
 1   releaseDate     1500 non-null   object 
 2   copiesSold      1500 non-null   int64  
 3   price           1500 non-null   float64
 4   revenue         1500 non-null   float64
 5   avgPlaytime     1500 non-null   float64
 6   reviewScore     1500 non-null   int64  
 7   publisherClass  1500 non-null   object 
 8   publishers      1499 non-null   object 
 9   developers      1498 non-null   object 
 10  steamId         1500 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 129.0+ KB


In [4]:
steamID_list = [id for id in steam_df["steamId"]]

In [5]:
import requests

In [6]:
import time

In [ ]:
appid = '1623730'  # Example App ID for Palworld
api_key = '792F552F899E56E815A791C06563F031'  # Replace with your API key

In [7]:
def remove_text_in_triangle_brackets(string):
    list = []
    inside_brackets = False

    for char in string:
        if char == '<':
            inside_brackets = True
        elif char == '>':
            inside_brackets = False
        elif not inside_brackets:
            list.append(char)
    return ''.join(list)

In [22]:
def get_game_data(appid):
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}'
    response = requests.get(url)
    return response.json()

appdata = get_game_data(2315690)

appdata[str(2315690)]

{'success': True,
 'data': {'type': 'game',
  'name': 'WWE 2K24',
  'steam_appid': 2315690,
  'required_age': '16',
  'is_free': False,
  'dlc': [2768190,
   2855630,
   2855620,
   2528460,
   2855610,
   2768170,
   2544390,
   2584660,
   2768160,
   3180050],
  'detailed_description': '<h1>WWE 2K24 Bray Wyatt Edition</h1><p><p class="bb_paragraph"><img class="bb_img" src="https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/2315690/extras/WWE24-BRAY_WYATT_EDITION-METADATA-AGN-STEAM-EDITION_CUSTOM_IMAGE-616X353.jpg?t=1729086528" />Introducing the WWE 2K24 Bray Wyatt Edition - an exclusive release that celebrates the legacy of one of WWE’s most captivating Superstars. Known for his enigmatic personas. With his unique blend of charisma, in-ring prowess, and storytelling ability, Bray Wyatt captured the imagination of wrestling fans around the world. Whether portraying the eerie cult leader of The Wyatt Family, the mysterious host of the Firefly Fun House, or the terrifyi

In [30]:
import requests
import pandas as pd
import time

def get_game_data(appid):
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}'
    response = requests.get(url)
    return response.json()

game_data_list = []
i = 0
j = 0
k = 0

for appid in steamID_list:
    i += 1
    appid_str = str(appid)  # Ensure appid is a string
    appdata = get_game_data(appid_str)
    
    #print(f"Attempt {i}: {appid_str}")  # Print current attempt

    # Check if the API response contains the appid and if the success key exists
    if appid_str in appdata and appdata[appid_str].get('success'):
        j += 1
        print(f"\rAttempt {i}: {appid_str} Success. {k} Failures, {j} Successes. {100*i/len(steamID_list)}% complete", end="")
        data = appdata[appid_str].get('data', {})

        lang_string = data.get('supported_languages', None)
        clean_lang_string = remove_text_in_triangle_brackets(lang_string).replace("*", "").replace("languages with full audio support", "") if lang_string else None
        lang_list = [item.strip() for item in clean_lang_string.split(',')] if clean_lang_string else []

        game_data_list.append({
            'game_name': data.get('name', None),
            'steam_id': data.get('steam_appid', None),
            'required_age': data.get('required_age', None),
            'is_free': data.get('is_free', None),
            'description': data.get('short_description', None),
            'languages': lang_list,
            'developers': data.get('developers', None),
            'publishers': data.get('publishers', None),
            'release_date': data.get('release_date', None),
            'price': data.get('price_overview', {}).get('final_formatted', None),
            'metacritic_score': data.get('metacritic', {}).get('score', None),
            'genres': [genre['description'] for genre in data.get('genres', [])],
            'categories': [cat['description'] for cat in data.get('categories', [])]
        })
    else:
        # Log the entire response for debugging if it fails
        k += 1
        print(f"\rAttempt {i}: {appid_str} Failure. {k} Failures, {j} Successes. {100*i/len(steamID_list)}% complete", end="")
        
        #print(f"Response: {appdata}")  # Print the response to understand why it failed
        game_data_list.append({
            'game_name': None,
            'steam_id': appid_str,
            'required_age': None,
            'is_free': None,
            'description': None,
            'languages': None,
            'developers': None,
            'publishers': None,
            'release_date': None,
            'price': None,
            'metacritic_score': None,
            'genres': None,
            'categories': None
        })

    time.sleep(1.5)

print()
game_df = pd.DataFrame(game_data_list)
game_df.info()


Attempt 1500: Success 1498 for 1296610. Total 1498/1500
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   game_name         1498 non-null   object 
 1   steam_id          1500 non-null   object 
 2   required_age      1498 non-null   object 
 3   is_free           1498 non-null   object 
 4   description       1498 non-null   object 
 5   languages         1498 non-null   object 
 6   developers        1498 non-null   object 
 7   publishers        1496 non-null   object 
 8   release_date      1498 non-null   object 
 9   price             1411 non-null   object 
 10  metacritic_score  96 non-null     float64
 11  genres            1498 non-null   object 
 12  categories        1498 non-null   object 
dtypes: float64(1), object(12)
memory usage: 152.5+ KB


In [31]:
game_df.to_csv(r"..\data\raw\game_api_data.csv", index=False, sep=';', header=True)


In [47]:
game_df = pd.DataFrame(game_data_list)

game_df.head()

,game_name,steam_id,required_age,is_free,description,languages,developers,publishers,release_date,price,metacritic_score,genres,categories,release_date_2
0,WWE 2K24,2315690,16,False,Experience a gripping retelling of WrestleMani...,"[English, French, Italian, German, Spanish - S...",[Visual Concepts],[2K],"7 Mar, 2024",£24.99,NaN,"[Simulation, Sports]","[Single-player, Multi-player, PvP, Online PvP,...","7 Mar, 2024"
1,EARTH DEFENSE FORCE 6,2291060,0,False,Live the future of despair.The year is 2024 AD...,"[English, Traditional Chinese, Japanese, Korea...",[SANDLOT],[D3PUBLISHER],"25 Jul, 2024",£49.99,79.0,[Action],"[Single-player, Multi-player, Co-op, Online Co...","25 Jul, 2024"
2,Sins of a Solar Empire II,1575940,0,False,Battle for galactic dominance in this real-tim...,"[English, French, Italian, German, Japanese, K...","[Ironclad Games Corporation, Stardock Entertai...",[Stardock Entertainment],"15 Aug, 2024",£41.99,NaN,"[Simulation, Strategy]","[Single-player, Multi-player, PvP, Online PvP,...","15 Aug, 2024"
3,Legend of Mortal,1859910,0,False,The game world is set in a turbulent state and...,"[Traditional Chinese, Simplified Chinese]",[Obb Studio Inc.],"[Paras Games, Obb Studio Inc.]","14 Jun, 2024",£10.00,NaN,"[Adventure, Indie, RPG, Strategy]","[Single-player, Steam Achievements, Full contr...","14 Jun, 2024"
4,Shin Megami Tensei V: Vengeance,1875830,17,False,Become a mighty demigod and battle through a d...,"[English, French, Italian, German, Spanish - S...",[ATLUS],[SEGA],"13 Jun, 2024",CDN$ 55.99,90.0,"[Adventure, RPG]","[Single-player, Steam Achievements, Full contr...","13 Jun, 2024"


In [48]:
game_df2 = game_df.copy()

cols_to_convert = ['languages', 'developers', 'publishers', 'genres', 'categories']

for col in cols_to_convert:
    game_df2[col] = game_df2[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

game_df2.head()

,game_name,steam_id,required_age,is_free,description,languages,developers,publishers,release_date,price,metacritic_score,genres,categories,release_date_2
0,WWE 2K24,2315690,16,False,Experience a gripping retelling of WrestleMani...,"English, French, Italian, German, Spanish - Sp...",Visual Concepts,2K,"7 Mar, 2024",£24.99,NaN,"Simulation, Sports","Single-player, Multi-player, PvP, Online PvP, ...","7 Mar, 2024"
1,EARTH DEFENSE FORCE 6,2291060,0,False,Live the future of despair.The year is 2024 AD...,"English, Traditional Chinese, Japanese, Korean...",SANDLOT,D3PUBLISHER,"25 Jul, 2024",£49.99,79.0,Action,"Single-player, Multi-player, Co-op, Online Co-...","25 Jul, 2024"
2,Sins of a Solar Empire II,1575940,0,False,Battle for galactic dominance in this real-tim...,"English, French, Italian, German, Japanese, Ko...","Ironclad Games Corporation, Stardock Entertain...",Stardock Entertainment,"15 Aug, 2024",£41.99,NaN,"Simulation, Strategy","Single-player, Multi-player, PvP, Online PvP, ...","15 Aug, 2024"
3,Legend of Mortal,1859910,0,False,The game world is set in a turbulent state and...,"Traditional Chinese, Simplified Chinese",Obb Studio Inc.,"Paras Games, Obb Studio Inc.","14 Jun, 2024",£10.00,NaN,"Adventure, Indie, RPG, Strategy","Single-player, Steam Achievements, Full contro...","14 Jun, 2024"
4,Shin Megami Tensei V: Vengeance,1875830,17,False,Become a mighty demigod and battle through a d...,"English, French, Italian, German, Spanish - Sp...",ATLUS,SEGA,"13 Jun, 2024",CDN$ 55.99,90.0,"Adventure, RPG","Single-player, Steam Achievements, Full contro...","13 Jun, 2024"


In [50]:
game_df2.to_csv(r"..\data\raw\game_api_data.csv", index=False, sep=';', header=True)


In [51]:
steam_data_filepath = r"..\data\raw\game_api_data.csv"

steam_df = pd.read_csv(steam_data_filepath, sep=';')

steam_df.head()

,game_name,steam_id,required_age,is_free,description,languages,developers,publishers,release_date,price,metacritic_score,genres,categories,release_date_2
0,WWE 2K24,2315690,16.0,False,Experience a gripping retelling of WrestleMani...,"English, French, Italian, German, Spanish - Sp...",Visual Concepts,2K,"7 Mar, 2024",£24.99,NaN,"Simulation, Sports","Single-player, Multi-player, PvP, Online PvP, ...","7 Mar, 2024"
1,EARTH DEFENSE FORCE 6,2291060,0.0,False,Live the future of despair.The year is 2024 AD...,"English, Traditional Chinese, Japanese, Korean...",SANDLOT,D3PUBLISHER,"25 Jul, 2024",£49.99,79.0,Action,"Single-player, Multi-player, Co-op, Online Co-...","25 Jul, 2024"
2,Sins of a Solar Empire II,1575940,0.0,False,Battle for galactic dominance in this real-tim...,"English, French, Italian, German, Japanese, Ko...","Ironclad Games Corporation, Stardock Entertain...",Stardock Entertainment,"15 Aug, 2024",£41.99,NaN,"Simulation, Strategy","Single-player, Multi-player, PvP, Online PvP, ...","15 Aug, 2024"
3,Legend of Mortal,1859910,0.0,False,The game world is set in a turbulent state and...,"Traditional Chinese, Simplified Chinese",Obb Studio Inc.,"Paras Games, Obb Studio Inc.","14 Jun, 2024",£10.00,NaN,"Adventure, Indie, RPG, Strategy","Single-player, Steam Achievements, Full contro...","14 Jun, 2024"
4,Shin Megami Tensei V: Vengeance,1875830,17.0,False,Become a mighty demigod and battle through a d...,"English, French, Italian, German, Spanish - Sp...",ATLUS,SEGA,"13 Jun, 2024",CDN$ 55.99,90.0,"Adventure, RPG","Single-player, Steam Achievements, Full contro...","13 Jun, 2024"
